# AI Augmented Electron Microscope Spectroscopy

By "Aaron Brookhouse"

<img src="Mitochondria.png" alt="2D Mitochondria Raw Image, Man-made labels, and predicted Labels" width="100%">
In the above image, the Machine Learning algorithm achieved a pixel by pixel F1 Classification score of .89, but the percent difference in predicted mitochondria volumes between the model output and true output was around .003% This suggests that the model is about as likely to false positive as it is to false negative, which allows it to predict volume very precisely.


<img src="Mito3D.png" alt="3D Reconstruction Of Mitochondria Done with Machine Learning" width="100%">
In the image above, the model's output is used to create a 3d reconstruction of the cell's mitochondria.

<img src="CellWall.png" alt="3D Reconstruction Of Cell Walls" width="100%">
In the image above, a different model's output is being to create a 3d reconstruction of the cell's walls. Highlighted are tunnels between the cell walls used for transport

---
# Abstract
 
Currently I am working with a plant biology group at Washington State University. They are interested in using Machine Learning techniques to speed up their image labelling needs for their research, as this is very time consuming to do by hand. The machine learning model that I am using (A version of the 3D Residual Unet), learns from raw electron microscope images and man made labels to be able to label cells itself. 

Due to the 3D nature of this image data, and model output data, arrays can get big in size very quickly. For example, a 500 deep image stack of 1920 by 1080 images has over a billion pixels. When preprocessing the man made labels (which are in image form) into the machine readable form, or postprocessing the raw machine learning model output, the involved arrays can be huge, and simple tasks can take a lot of time. It can be annoying when you want to change one parameter in the postprocessing to see how it affects the output and then you must wait a while. For this reason, I want to try a few different ways to manipulate a large array to see which one is the best. Once I know the best way to do one simple array modification, I will try it on other manipulations at some point in the future. I tried the array operation of thresholding (if an array location was above threshold set it to one, else set it to zero) using python lists, vectorized numpy, cupy (a cuda numpy library), and my own cuda code written in C. Unsurprisingly the cuda options were the fastest, and I was able to write C code that could handle larger arrays in Cuda than cupy was able to.

The above goal is the one that most relates to this class as I will be using some of the parallelization methods we discussed in class. I have some other objectives with this project as well.

*Create a GUI program to make some of the Machine Learning Techniques more easily accessable to people who are not comfortable with command line interfaces / using scripts and config files

*Get the AI stack running on the HPCC cluster, so people with out big GPUs can still use the AI program

---
# Methodology

I included the file _installPythonDependancies.sh which will load the needed modules, create a virtual environment, install dependancies, and then run the c and python programs


The main algorythm that I am benchmarking and trying different metods to compare is a thresholding algorythm. The machine learning outputs a fractional number representing confidence in it's output as a label for each pixel. To be usable, this must be turned into a binary yes/no classification. In words, my simple thresholding algorythm is if an array location was above threshold set it to one, else set it to zero. I created four versions of this algorythm:

*First, I used python lists, and looped through each element of the array to do the thresholding.

*Second, I used numpy with masking to do thresholding (array[array > threshold] = 1)

*Third, I exchanged numpy for cupy, which is a python library that is compatible with numpy (meaning the methods from a user perspective are the same)

*Fourth, I wrote a custom CUDA program in C to address some problems I had with Cupy.


To test and benchmark these different versions, I wrote a python program. However, I ran into one issue with cupy that was a bit frusturating. Once you got around 1.4 billion array elements (which is only about 1.4 gigabytes when using the int8 datatype) I would get errors from cupy saying I couldn't allocate enough memory to the graphics card. This doesn't make any sense, as I was using this on a graphics card that has 11G of VRAM. My guess is that since Cupy is made to be very generalized and work for many different situations, it may use more memory than actually needed, and it probably allocates additional memory to do things like array masking and such which I was using for my thresholding algorythm. 

To try to extend this upper limit of array size, I wrote a CUDA program in C to do the thresholding algorythm. The hope was that this could be more streamlined and specific to my task. For the python program I had used real data, however in c to make it easier I used random data with the same range of values. The C program for different array sizes and several trials generates a random array of that size, and allocates the correct amount of Device memory. It times how long it takes to copy the memory from host to device, make the calculation, and then copy the modified array from device back to host.

To run the C program, run the following two commands

* nvcc cudaBenchmark.cu -o cudaBenchmark

* ./cudaBenchmark

The program will print out at the end array sizes used, and then average time for those sizes. If you want, you can copy this information into the python program PythonBenchmark.py at the bottom and it will include it in the graph. Otherwise, my testing data is hardcoded as that was the easiest way to transfer from c to python.

To run the python program, you need to be in an environment with the prerequisite libraries installed, and all you need to do is run python3 PythonBenchmark.py.

 

---
# Timing Results


Hardware Used:

6 Core 3.6 GHz AMD Processor

32 G Ram

NVIDIA GTX 1080 TI GPU (11G VRAM)



<img src="RenumberList.png" width="50%">

The above graph shows how 3 versions of the thresholding algorythm scale as I increase the size of the 3D array. The number of total array elements is the x axis, and average time to run the algorythm over several trials is shown on the Y axis. It is clear that the version using python lists is severly unoptomized, and that using vectorized numpy and cupy is much faster.

<img src="Renumber.png" width="50%">

This graph removes the list version from the graph, as it is signifigantly slower and makes it harder to read the difference between the other versions of the algorythm. With this version of the graph you can really see how much faster the cupy version is than the numpy version. Here, you can see the cupy stops at around 1.4 billion elements due to the issue mentioned above. For this graph I ran the Numpy and Cupy arrays up to 1.4 billion elements (the numpy doesn't have the same issue, I just didn't test it longer as it showed a clear, linear trend). My cuda version is very comperable to cupy in regards to time, and by using the char datatype (1 byte per array element) I was able to run the thresholding algorythm on more than 1.4 billion elements. At 4 billion elements, my program was still about the same speed as numpy was at 1 billion, so I consider the custom cupy program a success.

---
# Concluding Discussion and Future Work

Overall, I definately did find a way to make the array operations for thresholding faster than a naive way with loops and also faster than numpy using vectorized masks. While 7x speedup from numpy to cupy may not seem super signifigant when the difference is 1.4s to .2s, this actually will be super helpful. In the future my group wants to work with bigger and bigger datasets, so the difference will become bigger as the source arrays get bigger. Also, this thresholding is used in a part of a different program where the raw data is thresholded into a binary array over and over again as the program sweeps the threshold. So if 100 thresholds are tested, this is actually the difference between 140 seconds and 20 seconds, which seems more signifigant.

In the future, I plan to take my findings with cupy and integrate them to other parts of my work. There are several ways I routinely do large array operations that I believe cupy has the potential to scale up. It may even be beneficial for me to use a custom cuda program like I did above and wrap it in python for some of them as this helped me get around some limitations of cupy.


There is also another program I am working on, that is still a work in progress that is attatching a GUI to the machine learning training and also interfacing with the HPCC. The program can train models, predict, and visualize created .ply files in an interactive window (showing 3d models like the one at the top of the report). A GUI is required to use it, but I have included it in the github anyway in case you are interested. I will attatch some screenshots below. The program can be ran with the command python3 tabbedGUI.py It currently is not completely finished, and some screens are still making some assumptions or have something hardcoded in that I need to generalize. Some of the buttons do not actually attatch to code yet, as I am currently more focused on improving the code that uses the AI and the AI's outputs, and designing the application to be the most general purpose and simple to use.


<img src="guiTrain.png" width="50%">

This screen has many options for configuring the network. The most important ones are to select the input images and the labels. You can train the network with the train button or if you put in your username and password it will connect to the hpcc and run a submission script there. Right now some of that is hardcoded to run a submission script on my account. However, I will be making it soon so it makes a custom submission script based on the inputs on this screen that it will then copy to the hpcc and run.

<img src="guiPredict.png" width="50%">

This screen has tools to run the model on images other than the training image to produce a raw model output file as an .h5 file

<img src="guiEvaluate.png" width="50%">

This screen is in its rudimentary stages, but it sweeps thresholding (currently only works if the model only has one output plane) through a certain range and outputs a graph showing how the accuracy, precision, recall, f1, and percent difference in volume change as a function of the threshold. Not very robust yet and makes some assumptions, needs to be made more variable.

<img src="guiOutputTools.png" width="50%">

This screen allows you to select a model output .h5 file and create point clouds and meshes with it, these clouds and meshes (saved as .ply) can be opened in an interactive 3d visualizer in the next screen.

<img src="guiVisualize.png" width="50%">

This screen allows you to pick .ply files to visualize

---
# References

&#9989;  Include links to websites and resources used in this project.  

The following are two github projects I have used extensively for my project and other parts of my work not in this report. The first is a machine learning stack from Harvard's Visual Computing group, and the second is a 3D python graphics library from Intel.

https://github.com/zudi-lin/pytorch_connectomics

https://github.com/intel-isl/Open3D

-----
### Congratulations, you are done!

Now, you just need to create a second directory in your git repository and include your report as an md or ipynb file in the directory along with any additional figures and files needed to reproduce the results.  You instructor should already have your git repository and be able to pull in your changes. 

Written by Dr. Dirk Colbry, Michigan State University
<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.

----